O pacote python-bcb implementa interfaces para algumas APIs disponibilizadas pelo Banco Central de forma que o resultado das consultas, na maioria dos casos, é um DataFrame pandas formatado com os dados.


APIs implementadas: SGS (Sistema Gerenciador de Séries Temporais), Conversor de Moedas, Moedas OData, Expectativas, Taxas de Juros e demais APIs

In [113]:
from bcb import sgs, currency,PTAX,Expectativas,TaxaJuros,ODataAPI

In [122]:
df = sgs.get({'Novo Caged - Total': 28763}, start='2002-02-01')

df.index = df.index.to_period('M')

df.head()

,Novo Caged - Total
Date,
2002-02,22306062
2002-03,22396322
2002-04,22571412
2002-05,22727225
2002-06,22860571


EntitySets:
  ExpectativasMercadoTop5Anuais
  ExpectativaMercadoMensais
  ExpectativasMercadoInflacao24Meses
  ExpectativasMercadoInflacao12Meses
  ExpectativasMercadoSelic
  ExpectativasMercadoTop5Selic
  ExpectativasMercadoTop5Mensais
  ExpectativasMercadoTrimestrais
  ExpectativasMercadoAnuais


In [91]:
em.describe('ExpectativasMercadoTop5Anuais')


EntitySet (Endpoint): ExpectativasMercadoTop5Anuais
EntityType: br.gov.bcb.olinda.servico.Expectativas.ExpectativasMercadoTop5Anual
Properties: Indicador<str>, Data<str>, DataReferencia<str>, tipoCalculo<str>, Media<float>, Mediana<float>, DesvioPadrao<float>, Minimo<float>, Maximo<float>


In [92]:
ep = em.get_endpoint('ExpectativasMercadoTop5Anuais')
ep.query().limit(10).collect()

,Indicador,Data,DataReferencia,tipoCalculo,Media,Mediana,DesvioPadrao,Minimo,Maximo
0,PIB Total,2023-03-31,2023,L,1.5936,1.7004,0.3553,0.9700,2.00
1,PIB Total,2023-03-31,2024,L,0.9817,1.1000,0.4276,0.4585,1.45
2,PIB Total,2023-03-31,2025,L,1.5833,1.5000,0.1546,1.4500,1.80
3,PIB Total,2023-03-31,2026,L,1.5833,1.5000,0.1546,1.4500,1.80
4,PIB Total,2023-03-31,2027,L,1.5833,1.5000,0.1546,1.4500,1.80
5,PIB Total,2023-04-03,2023,L,1.5936,1.7004,0.3553,0.9700,2.00
6,PIB Total,2023-04-03,2024,L,0.9817,1.1000,0.4276,0.4585,1.45
7,PIB Total,2023-04-03,2025,L,1.5833,1.5000,0.1546,1.4500,1.80
8,PIB Total,2023-04-03,2026,L,1.5833,1.5000,0.1546,1.4500,1.80
9,PIB Total,2023-04-03,2027,L,1.5833,1.5000,0.1546,1.4500,1.80


In [93]:
ep2 = Expectativas().get_endpoint('ExpectativasMercadoTop5Anuais').query().collect()

ep2.groupby("Indicador").agg({"Media": "median"})

,Media
Indicador,
Câmbio,3.0500
IGP-DI,4.7500
IGP-M,4.6400
IPCA,4.4800
IPCA Administrados,3.8566
PIB Total,1.8667
Selic,10.0000
Taxa de desocupação,8.6667


In [96]:
ep.query().filter(ep.Indicador == 'IPCA').collect()
(ep.query()
 .filter(ep.Indicador == 'IPCA', ep.DataReferencia == 2024)
 .filter(ep.Data >= '2022-01-01')
 .filter(ep.tipoCalculo == 'C')
 .select(ep.Indicador,ep.Data, ep.Media, ep.Mediana)
 .orderby(ep.Data.desc())
 .limit(100)
 .collect())

,Indicador,Data,Media,Mediana
0,IPCA,2024-05-03,3.8521,3.6162
1,IPCA,2024-05-02,3.8335,3.5851
2,IPCA,2024-04-30,3.8335,3.5851
3,IPCA,2024-04-29,3.8335,3.5851
4,IPCA,2024-04-26,3.8438,3.5851
...,...,...,...,...
95,IPCA,2023-12-14,3.8739,3.9282
96,IPCA,2023-12-13,3.8739,3.9282
97,IPCA,2023-12-12,3.8739,3.9282
98,IPCA,2023-12-11,3.8300,3.7400
